# Exploring `add_electricity.py` outputs

This jupyter notebook analyse the `add_electricity.py` **outputs**.

The `pypsa-africa/Snakefile` explicitly list in the **rule** what goes into the function `add_electricity.py` and what goes out (`elec.nc`). When inputs are commented out (#) than they are currently not considered. Maybe you want to make them run or develop new features? What can be observed from the inputs is that the script adds electrical generators, existing hydro storage, renewable generator profiles and loads units to a base network topology.

```
rule add_electricity:
    input:
        base_network='networks/base.nc',
        tech_costs=COSTS,
        regions="resources/regions_onshore.geojson",
        powerplants='resources/powerplants.csv',
        # hydro_capacities='data/bundle/hydro_capacities.csv',
        # geth_hydro_capacities='data/geth2015_hydro_capacities.csv',
        load='resources/ssp2-2.6/2030/era5_2013/Africa.nc',
        gadm_shapes='resources/gadm_shapes.geojson',
        **{f"profile_{tech}": f"resources/profile_{tech}.nc"
            for tech in config['renewable']}
    output: "networks/elec.nc"
    log: "logs/add_electricity.log"
    benchmark: "benchmarks/add_electricity"
    threads: 1
    resources: mem=3000
    script: "scripts/add_electricity.py"
```

Before analysing the outputs of add_electricity.py check that:
- `pypsa-africa` environment (/kernel) in jupyter notebook  is active and updated
- root folder where pypsa-africa is installed is named "pypsa-africa"
- or rename the below `_sets_path_to_root("<folder_name>")` accordingly

In [1]:
import sys
sys.path.append('../')  # to import helpers
from scripts._helpers import _sets_path_to_root
_sets_path_to_root("pypsa-africa")

A jupyter notebook requires the user to import all they need. So we need to import all the required dependencies from the `pypsa-africa` environment:

In [2]:
import logging
import os

import pypsa
import yaml
import pandas as pd
import geopandas as gpd
#import geoviews as gv
#import hvplot.pandas
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib as plt

from scipy.sparse import csgraph
from itertools import product

from shapely.geometry import Point, LineString
import shapely, shapely.prepared, shapely.wkt

logger = logging.getLogger(__name__)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 70)

## Network analysis

Now let's import the output file which should be in i.e. `~/pypsa-africa/networks/elec.nc`. 

Note: If you don't have the /elec.nc file yet in your folder structure, you could try to generate it from the terminal. In this case set the path so that you are located at `~/pypsa-africa`. Once you are there, use the command:
```
snakemake -j 1 networks/elec.nc
```
Note that this is something you can to for any other rule to generate it's outputs `snakemake -j 1 <output of rule>`

In [3]:
network_path = os.getcwd() + "/networks/elec.nc"
n = pypsa.Network(network_path)
n

You might be interested to see what's in the Network Common Data Form or NetCDF (.nc) file. To assess that we use **xarray**

In [5]:
import xarray as xr

path = os.getcwd() + "/networks/elec.nc"
network_data = xr.open_dataset(path)

network_data

A quick check of the network topology:

In [7]:
n.plot()

A full list of components are given: https://pypsa.readthedocs.io/en/latest/components.html#
We always check the PyPSA documentation to find analyse the components of interest.

In [6]:
n.buses.head(4)

In [5]:
n.lines.head(2)

In [11]:
# The maximal available power output
n.generators.groupby("carrier").p_nom_max.sum() / 1e3 # converted in [GW]

In [13]:
# The (by the model) optimised installed power 
n.generators.groupby("carrier").p_nom_opt.sum() / 1e3  # converted in [GW]

### Maximal available wind profile (as p.u.)

A time series is of the type of "series". See: https://pypsa.readthedocs.io/en/latest/components.html#generator
This requires to use `n.generator_t.p_max_pu` instead `n.generator.p_max_pu`

In [14]:
n.generators_t.p_max_pu.iloc[:,0].plot()

# Load timeseries


In [20]:
n.loads_t.p